In [1]:
import asyncio
import json
import re
from urllib.request import Request, urlopen

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
f = open("./data/articles.json", "r")
txt = f.read()

def removeNonAscii(s):
    return "".join(i for i in s if ord(i)<128)

data = json.loads(removeNonAscii(txt))
print(str(data)[:500] + ' ...')

{'rss': {'channel': {'title': 'Daily Bruin', 'link': 'https://dailybruin.com', 'description': 'Since 1919', 'pubDate': 'Thu, 25 Jun 2020 03:41:48 +0000', 'language': 'en-US', 'wxr_version': {'__text': '1.2'}, 'base_site_url': {'__text': 'https://dailybruin.com'}, 'base_blog_url': {'__text': 'https://dailybruin.com'}, 'author': [{'author_id': {'__text': '6658'}, 'author_login': {'__cdata': 'Camayak_0d6b9a1acc'}, 'author_email': {'__cdata': 'wordpress+0d6b9a1acc@camayak.com'}, 'author_display_name ...


In [3]:
raw_xml_df = pd.DataFrame.from_dict(data['rss']['channel']['item']).drop(columns = ["comment_status", "description", "guid", "is_sticky", "menu_order", "ping_status", "post_parent", "post_password", "post_id", "post_type", "postmeta", "status", "post_name", "post_date", "post_date_gmt", "comment"])
raw_xml_df.head()

,title,link,pubDate,creator,encoded,category
0,Baseball puts up strong showing with season-hi...,https://dailybruin.com/2020/03/01/baseball-put...,"Sun, 01 Mar 2020 09:12:10 +0000",{'__cdata': 'Camayak_d98b46ec94'},[{'__cdata': 'The Bruins faced off against one...,"[{'_domain': 'author', '_nicename': 'cap-will-..."
1,Gallery: UCLA mens basketball ekes out a victo...,https://dailybruin.com/2020/03/01/gallery-ucla...,"Sun, 01 Mar 2020 22:35:46 +0000",{'__cdata': 'Liz Ketcham'},"[{'__cdata': '<span style=""font-weight: 400;"">...","[{'_domain': 'author', '_nicename': 'cap-liz-k..."
2,No. 1 UCLA beach volleyball falls to No. 2 LSU...,https://dailybruin.com/2020/03/01/no-1-ucla-be...,"Mon, 02 Mar 2020 00:04:17 +0000",{'__cdata': 'Camayak_7035d234b0'},[{'__cdata': 'The Tigers have been the team to...,"[{'_domain': 'author', '_nicename': 'cap-lexi-..."
3,Men's volleyball falls short in five-set battl...,https://dailybruin.com/2020/03/01/mens-volleyb...,"Mon, 02 Mar 2020 00:51:14 +0000",{'__cdata': 'Camayak_f18f6258dd'},[{'__cdata': '<p>The Bruins were a few points ...,"[{'_domain': 'category', '_nicename': 'mens-vo..."
4,Track and field notches top-5 finishes in men'...,https://dailybruin.com/2020/03/01/track-and-fi...,"Mon, 02 Mar 2020 01:57:17 +0000",{'__cdata': 'Camayak_af38495c87'},[{'__cdata': '<p>In their first championship m...,"[{'_domain': 'category', '_nicename': 'sports'..."


In [4]:
def parse_categories(items):
    authors = []
    categories = []
    for d in items:
        if d['_domain'] == 'author':
            authors.append(d['_nicename'])
        if d['_domain'] == 'category':
            categories.append(d['_nicename'])
    return authors, categories


def parse_db_url_path(url):
    regex = r"https://(?:.*.)?dailybruin.com(\/.*)"
    matches = re.match(regex, url)
    if matches is None:
        raise ValueError(f"[parse_db_url_path] - Invalid URL - {url}")
    return matches.group(1)


xml_df = raw_xml_df
xml_df.creator = xml_df.creator.map(lambda x: x['__cdata'])
xml_df.encoded = xml_df.encoded.map(lambda x: x[0]['__cdata'])

authors_and_categories = xml_df.category.map(parse_categories)
xml_df['authors'] = [a for a, _ in authors_and_categories]
xml_df['categories'] = [c for _, c in authors_and_categories]

xml_df['page'] = xml_df.link.map(parse_db_url_path)

xml_df = xml_df.drop('category', axis=1)
xml_df = xml_df.rename(columns={'encoded': 'encoded_text'})

xml_df.head()

,title,link,pubDate,creator,encoded_text,authors,categories,page
0,Baseball puts up strong showing with season-hi...,https://dailybruin.com/2020/03/01/baseball-put...,"Sun, 01 Mar 2020 09:12:10 +0000",Camayak_d98b46ec94,The Bruins faced off against one of their toug...,[cap-will-foote],"[baseball, sports]",/2020/03/01/baseball-puts-up-strong-showing-wi...
1,Gallery: UCLA mens basketball ekes out a victo...,https://dailybruin.com/2020/03/01/gallery-ucla...,"Sun, 01 Mar 2020 22:35:46 +0000",Liz Ketcham,"<span style=""font-weight: 400;"">Despite traili...","[cap-liz-ketcham, cap-jhan]","[spectrum, sports-spectrum]",/2020/03/01/gallery-ucla-mens-basketball-ekes-...
2,No. 1 UCLA beach volleyball falls to No. 2 LSU...,https://dailybruin.com/2020/03/01/no-1-ucla-be...,"Mon, 02 Mar 2020 00:04:17 +0000",Camayak_7035d234b0,The Tigers have been the team to beat for the ...,[cap-lexi-grimes],"[beach-volleyball, sports]",/2020/03/01/no-1-ucla-beach-volleyball-falls-t...
3,Men's volleyball falls short in five-set battl...,https://dailybruin.com/2020/03/01/mens-volleyb...,"Mon, 02 Mar 2020 00:51:14 +0000",Camayak_f18f6258dd,<p>The Bruins were a few points away from bein...,[],"[mens-volleyball, sports]",/2020/03/01/mens-volleyball-falls-short-in-fiv...
4,Track and field notches top-5 finishes in men'...,https://dailybruin.com/2020/03/01/track-and-fi...,"Mon, 02 Mar 2020 01:57:17 +0000",Camayak_af38495c87,<p>In their first championship meet of the sea...,[],"[sports, track-and-field]",/2020/03/01/track-and-field-notches-top-5-fini...


In [6]:
raw_pv_df = pd.read_csv('./data/page_views.csv')
raw_pv_df.head()

,Page,Pageviews,Unique Pageviews,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,/2020/06/04/students-sign-petition-to-fire-ucl...,"40,351","37,035",0:04:55,"36,124",90.77%,89.38%,$0.00
1,/2020/04/24/uc-and-ucla-leadership-in-discussi...,"28,584","27,159",0:04:03,"25,859",92.23%,90.34%,$0.00
2,/2020/04/10/stans-donuts-to-close-after-55-yea...,"18,392","17,048",0:03:17,"16,796",91.16%,89.90%,$0.00
3,/2020/04/28/ucla-announces-aim-to-offer-remote...,"15,642","14,510",0:03:21,"13,672",86.78%,84.82%,$0.00
4,/2020/05/13/ucla-tells-faculty-to-prepare-for-...,"14,876","13,723",0:03:54,"13,038",89.38%,87.39%,$0.00


In [7]:
def sanitize_page(page):
    if "?" in page:
        page = page.split("?")[0]
    if page[0] != "/":
        page = "/" + page
    if page[-1] != "/":
        page += '/'
    return page


pv_df = raw_pv_df.rename(
    columns={
        'Page': 'page',
        'Pageviews': 'views',
        'Unique Pageviews': 'unique_views',
        'Avg. Time on Page': 'avg_time',
        'Entrances': 'entrances',
        'Bounce Rate': 'bounce_rate',
        '% Exit': 'exit_percent',
        'Page Value': 'page_value',
    }
)

pv_df.page = pv_df.page.map(sanitize_page)

pv_df.head()

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value
0,/2020/06/04/students-sign-petition-to-fire-ucl...,"40,351","37,035",0:04:55,"36,124",90.77%,89.38%,$0.00
1,/2020/04/24/uc-and-ucla-leadership-in-discussi...,"28,584","27,159",0:04:03,"25,859",92.23%,90.34%,$0.00
2,/2020/04/10/stans-donuts-to-close-after-55-yea...,"18,392","17,048",0:03:17,"16,796",91.16%,89.90%,$0.00
3,/2020/04/28/ucla-announces-aim-to-offer-remote...,"15,642","14,510",0:03:21,"13,672",86.78%,84.82%,$0.00
4,/2020/05/13/ucla-tells-faculty-to-prepare-for-...,"14,876","13,723",0:03:54,"13,038",89.38%,87.39%,$0.00


In [8]:
def parse_percent(s):
    return float(str(s).strip('%')) / 100


def parse_float(s):
    return float(str(s).replace(",", ""))


def parse_seconds(time):
    if str(time) == 'nan' or str(time) == 'None':
        return ''
    mins = int(str(time)[2:4])
    second = int(str(time)[5:])
    total_seconds = mins * 60 + second
    return total_seconds


def clean_html(html):
    html = re.sub(r'<.*?>', ' ', str(html))
    html = re.sub(r'\n', ' ', html)
    html = re.sub(r'\\', ' ', html)
    return html


df = pd.merge(
    pv_df, xml_df, how="outer", left_on="page", right_on="page"
)

df.exit_percent = df.exit_percent.map(parse_percent)
df.bounce_rate = df.bounce_rate.map(parse_percent)
df.views = df.views.map(parse_float)
df.unique_views = df.unique_views.map(parse_float)
df.entrances = df.entrances.map(parse_float)
df.avg_time = df.avg_time.map(parse_seconds)
df.encoded_text = df.encoded_text.map(clean_html)

df.head()

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories
0,/2020/06/04/students-sign-petition-to-fire-ucl...,40351.0,37035.0,295,36124.0,0.9077,0.8938,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
1,/2020/06/04/students-sign-petition-to-fire-ucl...,463.0,395.0,290,391.0,0.8670,0.8488,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
2,/2020/06/04/students-sign-petition-to-fire-ucl...,151.0,135.0,228,127.0,0.7244,0.7152,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
3,/2020/06/04/students-sign-petition-to-fire-ucl...,92.0,79.0,328,75.0,0.8400,0.8370,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
4,/2020/06/04/students-sign-petition-to-fire-ucl...,71.0,61.0,193,61.0,0.8689,0.8451,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"


In [9]:
df = df.drop_duplicates(subset=['page']).dropna()
df = df.reset_index(drop=True)
df['image_url'] = None

df.head()

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories,image_url
0,/2020/06/04/students-sign-petition-to-fire-ucl...,40351.0,37035.0,295,36124.0,0.9077,0.8938,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]",None
1,/2020/04/24/uc-and-ucla-leadership-in-discussi...,28584.0,27159.0,243,25859.0,0.9223,0.9034,$0.00,UC and UCLA leadership in discussion regarding...,https://dailybruin.com/2020/04/24/uc-and-ucla-...,"Sat, 25 Apr 2020 00:18:35 +0000",Camayak_3289e55774,This post was updated April 25 at 12:50 p.m. ...,[cap-megana-sekar],"[academics-news, campus, higher-education, new...",None
2,/2020/04/10/stans-donuts-to-close-after-55-yea...,18392.0,17048.0,197,16796.0,0.9116,0.8990,$0.00,Stans Donuts to close after 55 years of busine...,https://dailybruin.com/2020/04/10/stans-donuts...,"Fri, 10 Apr 2020 07:13:32 +0000",Camayak_0546053406,"Stans Donuts, a 55-year staple of Westwood Vi...",[cap-martin-bilbao],"[news, westwood]",None
3,/2020/04/28/ucla-announces-aim-to-offer-remote...,15642.0,14510.0,201,13672.0,0.8678,0.8482,$0.00,UCLA announces aim to offer remote instruction...,https://dailybruin.com/2020/04/28/ucla-announc...,"Wed, 29 Apr 2020 02:12:30 +0000",Camayak_43fdcc6551,UCLA plans to give students the option to take...,[cap-julia-shapero],"[campus, news]",None
4,/2020/05/13/ucla-tells-faculty-to-prepare-for-...,14876.0,13723.0,234,13038.0,0.8938,0.8739,$0.00,UCLA tells faculty to prepare for remote fall ...,https://dailybruin.com/2020/05/13/ucla-tells-f...,"Thu, 14 May 2020 03:00:48 +0000",Camayak_55924fa9bd,Faculty should expect large courses to be off...,[cap-bernard-mendez],"[academics-news, news]",None


In [11]:
def get_image_url(html):
    soup = BeautifulSoup(html)
    images = soup.findAll('meta', {'property': 'og:image'})
    if len(images) == 1:
        image = images[0]['content']
        return image
    else:
        print(f'Can\'t find image: {row.link}')

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
}

batch_size = 16

# Loops through batches of batch_size pages and scrapes images for them asynchronously
for i in range(0, df.shape[0], batch_size):
    print(f'Processing batch {i // batch_size} of {df.shape[0] // batch_size}')
    subset = df.iloc[i:i + batch_size]

    loop = asyncio.get_event_loop()
    requests = {
        'indices': [],
        'futures': [],
    }

    for j, row in subset.iterrows():
        if row.image_url is None:
            req = Request(row.link, headers=headers)
            future = loop.run_in_executor(None, urlopen, req)
            requests['indices'].append(j)
            requests['futures'].append(future)

    for j, res in zip(requests['indices'], await asyncio.gather(*requests['futures'])):
        html = res.read()
        df.image_url[j] = get_image_url(html)

Processing batch 0 of 51
Processing batch 1 of 51
Processing batch 2 of 51
Processing batch 3 of 51
Processing batch 4 of 51
Processing batch 5 of 51
Processing batch 6 of 51
Processing batch 7 of 51
Processing batch 8 of 51
Processing batch 9 of 51
Processing batch 10 of 51
Processing batch 11 of 51
Processing batch 12 of 51
Can't find image: https://dailybruin.com/2020/04/07/reports-of-domestic-violence-other-crimes-fall-in-la-following-stay-at-home-orders/
Processing batch 13 of 51
Processing batch 14 of 51
Processing batch 15 of 51
Processing batch 16 of 51
Processing batch 17 of 51
Processing batch 18 of 51
Processing batch 19 of 51
Processing batch 20 of 51
Processing batch 21 of 51
Processing batch 22 of 51
Processing batch 23 of 51
Processing batch 24 of 51
Processing batch 25 of 51
Processing batch 26 of 51
Processing batch 27 of 51
Processing batch 28 of 51
Processing batch 29 of 51
Processing batch 30 of 51
Processing batch 31 of 51
Processing batch 32 of 51
Processing batch

In [16]:
df.to_csv("./data/merged.csv")